### Mount Google Drive

In [ ]:
from google.colab import drive

drive.mount("content")

### Unzip dataset from Google Drive to Colab's disk

In [ ]:
!ls content/MyDrive
!unzip content/MyDrive/skull_dataset.zip

### Install tensorflow-gpu 2.3.0
The same tensorflow, that's listed in Conda environment

In [ ]:
!pip install -U tensorflow-gpu==2.3.0

In [ ]:
import tensorflow as tf
from tensorflow import keras
from pathlib import Path

### Install segmentation_models library

In [ ]:
# https://github.com/qubvel/segmentation_models/issues/374#issuecomment-672694688
%env SM_FRAMEWORK=tf.keras

In [ ]:
pip install segmentation-models

### Data generator

In [ ]:
from segmentation_models import get_preprocessing


def scans_generator(scans_train_directory_path: Path, scans_val_directory_path: Path, batch_size=16,
                    target_size=(256, 256), shuffle=True, backbone="efficientnetb0"):
    """
    Tensorflow dataset generator.

    :param scans_train_directory_path: It should contain one subdirectory per class. Any PNG, JPG, BMP, PPM or TIF
    images inside each of the subdirectories directory tree will be included in the generator.
    :param scans_val_directory_path: It should contain one subdirectory per class. Any PNG, JPG, BMP, PPM or TIF
    images inside each of the subdirectories directory tree will be included in the generator.
    :param backbone: backbone name of network you will use
    :param shuffle: Whether to shuffle the data (default: True) If set to False, sorts the data in alphanumeric order.
    :param target_size: The dimensions to which all images found will be resized.
    :param batch_size: Size of the batches of data
    :return train_combined_generator: A DirectoryIterator yielding tuples of (x, y) where x is a numpy array containing
    a batch of images with shape (batch_size, *target_size, channels) and y is a numpy array of corresponding labels
    :return val_combined_generator: A DirectoryIterator yielding tuples of (x, y) where x is a numpy array containing
    a batch of images with shape (batch_size, *target_size, channels) and y is a numpy array of corresponding labels
    :return train_samples: number of training samples in dataset
    :return val_samples: number of validation samples in dataset
    """

    # Preprocess image based on backbone implementation
    preprocess_input = get_preprocessing(backbone)

    images_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        # rescale=1/255, # -> Uncomment if not using preprocess_input
        preprocessing_function=preprocess_input
    )

    masks_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

    train_images_generator = images_datagen.flow_from_directory(directory=scans_train_directory_path / "scans",
                                                                target_size=target_size,
                                                                batch_size=batch_size,
                                                                seed=42,
                                                                shuffle=shuffle,
                                                                class_mode=None,
                                                                interpolation="bilinear",
                                                                )

    train_masks_generator = masks_datagen.flow_from_directory(directory=scans_train_directory_path / "labels",
                                                              target_size=target_size,
                                                              batch_size=batch_size,
                                                              seed=42,
                                                              shuffle=True,
                                                              class_mode=None,
                                                              interpolation="nearest",
                                                              )

    val_images_generator = images_datagen.flow_from_directory(directory=scans_val_directory_path / "scans",
                                                              target_size=target_size,
                                                              batch_size=batch_size,
                                                              seed=42,
                                                              shuffle=shuffle,
                                                              class_mode=None,
                                                              interpolation="bilinear",
                                                              )

    val_masks_generator = masks_datagen.flow_from_directory(directory=scans_val_directory_path / "labels",
                                                            target_size=target_size,
                                                            batch_size=batch_size,
                                                            seed=42,
                                                            shuffle=True,
                                                            class_mode=None,
                                                            interpolation="nearest",
                                                            )

    train_combined_generator = zip(train_images_generator, train_masks_generator)
    val_combined_generator = zip(val_images_generator, val_masks_generator)

    train_samples = train_images_generator.n
    val_samples = val_images_generator.n

    return train_combined_generator, val_combined_generator, train_samples, val_samples


### Getting Unet model from segmentation_models library

In [ ]:
from segmentation_models import Unet


def get_unet_model(backbone="efficientnetb0", classes=1, activation="sigmoid", encoder_weights="imagenet"):
    """
    Returns Unet model based on input variables

    :param backbone: Unet backbone
    :param classes: Number of classes
    :param activation: Activation function
    :param encoder_weights: Encoder weights
    :return model: Model based on input variables
    """
    model = Unet(backbone, classes=classes, activation=activation, encoder_weights=encoder_weights)

    return model


### Training input parameters

In [ ]:
train_path = Path("skull_dataset/train/x")
val_path = Path("skull_dataset/val/x")

backbone = "efficientnetb0"

batch_size = 32


### Loss and Metrics


In [ ]:
import segmentation_models as sm
from segmentation_models.losses import dice_loss
metrics = [sm.metrics.FScore(threshold=0.5)]

### Training


In [ ]:
train_generator, val_generator, train_samples, val_samples = scans_generator(train_path,
                                                                                val_path,
                                                                                backbone=backbone,
                                                                                batch_size=batch_size)

In [ ]:
model = get_unet_model(backbone=backbone, classes=1, activation="sigmoid", encoder_weights="imagenet")
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-3),
    loss=dice_loss,
    metrics=[metrics]
)

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_f1-score",
        patience=2,
        verbose=1,
        mode="max"
    ),
    keras.callbacks.ModelCheckpoint(
        monitor="val_f1-score",
        save_best_only=1,
        verbose=1,
        mode="max",
        filepath="model_{epoch}"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_f1-score",
        factor=0.5,
        patience=2,
        mode="max"
    )
]

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_samples//batch_size,
    validation_data=val_generator,
    validation_steps=val_samples//batch_size*2,
    callbacks=callbacks,
    epochs=20,
    verbose=1
)